# Import all the existing libraries and dataset

In [1]:
import numpy as np
import pandas as pd

In [11]:
csv_path = "./Data_Annotations.csv"
source_df = pd.read_csv(csv_path)

In [12]:
source_df.head()

,Name of file,Indendation,Indendation Consistent?,Whitespace around code to improve readability?,Used == instead of equals,Single/Nondescriptive variable names,Comments,Empty Catch Block,Generic Exceptions,Explicit Initialization,Method Length,Magic Numbers,# of magic numbers,Line Length,Boolean Exp Complexity,"FINAL CODE QUALITY METRIC (Good, Very Good, Decent, Bad, Very Bad)"
0,Snake.java,4,Yes,Yes,NaN,2,0,NaN,NaN,No,6,No,0,25,NaN,Bad
1,TimeTest.java,2,Yes,No,NaN,6,0,NaN,NaN,Yes,9,No,0,70,NaN,Bad
2,HelloWorld.java,9,No,No,NaN,1,1,NaN,NaN,Yes,6,Yes,1,40,NaN,Good
3,NoDisconnectManager.java,2,Yes,Yes,NaN,0,5,No,No,Yes,48,No,0,115,1,Good
4,TestIntegerNull.java,2,Yes,Yes,Yes,0,0,NaN,NaN,Yes,9,No,0,40,1,Bad


,Name of file,Indendation,Indendation Consistent?,Whitespace around code to improve readability?,Used == instead of equals,Single/Nondescriptive variable names,Comments,Empty Catch Block,Generic Exceptions,Explicit Initialization,Method Length,Magic Numbers,# of magic numbers,Line Length,Boolean Exp Complexity,"FINAL CODE QUALITY METRIC (Good, Very Good, Decent, Bad, Very Bad)"
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,NoDisconnectManager.java,2,Yes,Yes,NaN,0,5,No,No,Yes,48,No,0,115,1,Good
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,CheckContent.java,4,Yes,Yes,No,0,2,No,NaN,Yes,16,No,0,92,1,Good
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
